In [225]:
# Packages 
import pandas as pd

In [226]:
def ts_filter(df):
    """Filter TS numbers that appear in exactly 10 unique years."""
    df['ÅR'] = df['ÅR'].astype(str)
    ts_with_10_years = df.groupby('TS')['ÅR'].nunique()  
    valid_ts = ts_with_10_years[ts_with_10_years >= 5].index
    filtered_df = df[df['TS'].isin(valid_ts)]
    return filtered_df

In [227]:
# Merge retninger 
def merge_retninger(df):
    """Merge the retninger columns into a single column."""
    group_cols = ["TS", "VEJNAVN", "ÅR", "KATEGORI"]
    antal_cols = [col for col in df.columns if col.startswith("ANTAL")]
    df_grouped = df.groupby(group_cols, as_index=False)[antal_cols].sum()
    return df_grouped

In [228]:
# Rename Kateogri
def rename_categories(df, category_mapping):
    df['KATEGORI'] = df['KATEGORI'].replace(category_mapping)
    return df

In [229]:
# Merge Kategorier
def merge_and_sum_categories(df, source_category, target_category):
    count_columns = [col for col in df.columns if col.startswith("ANTAL")]
    source_rows = df[df["KATEGORI"] == source_category]
    target_rows = df[df["KATEGORI"] == target_category]

    for _, source_row in source_rows.iterrows():
        matching_target_row = target_rows[
            (target_rows["VEJNAVN"] == source_row["VEJNAVN"])
            & (target_rows["TS"] == source_row["TS"])
            & (target_rows["ÅR"] == source_row["ÅR"])
        ]
        if not matching_target_row.empty:
            target_index = matching_target_row.index[0]
            for col in count_columns:
                df.at[target_index, col] += source_row[col]

    df = df[df["KATEGORI"] != source_category]
    return df


In [230]:
# Total Sum
def Total_sum(df):
    antal_columns = [col for col in df.columns if col.startswith("ANTAL ")]
    df["TOTAL"] = df[antal_columns].sum(axis=1)
    return df

In [231]:
# Udregne tung trafik procent
def calculate_tung_traffic_percentage(df):
    Tunge = df[df["KATEGORI"] == "B: TUNG TRAFIK I ALT"].set_index(["TS", "ÅR"])["TOTAL"]
    def compute_percentage(row):
        if row["KATEGORI"] == "A+B: KØRETØJER I ALT":
            key = (row["TS"], row["ÅR"])
            if key in Tunge:
                return (Tunge[key] / row["TOTAL"]) if Tunge[key] != 0 else 0
        return None  # Keep other categories as NaN
    df["TUNG %"] = df.apply(compute_percentage, axis=1)
    
    return df

In [232]:
# Udregne Ladcykler i procent
def calculate_Ladcykler_percentage(df):
    Ladcykler = df[df["KATEGORI"] == "LADCYKLER"].set_index(["TS", "ÅR"])["TOTAL"]
    def compute_percentage(row):
        if row["KATEGORI"] == "CYKLER + KNALLERTER I ALT":
            key = (row["TS"], row["ÅR"])
            if key in Ladcykler:
                return (Ladcykler[key] / row["TOTAL"]) if Ladcykler[key] != 0 else 0
        return None  # Keep other categories as NaN
    df["LADCYKLER %"] = df.apply(compute_percentage, axis=1)
    
    return df

In [233]:
def main(input_file_path, output_file_path):
    df = pd.read_csv(input_file_path)
    df = ts_filter(df)
    df = merge_retninger(df)

    # Rename categories
    category_mapping = {
        "VAREVOGNE (MAX. 3,5 T)": "VAREVOGNE (MAX. 3,5 t)",
        "LASTBILER, 3 aksler": "LASTBILER, 3 AKSLER",
        "LASTBILER, 2 aksler": "LASTBILER, 2 AKSLER",
        "LASTBILER, 2-3 aksler": "LASTBILER, 2-3 AKSLER",
        "Ladcykler": "LADCYKLER"
    }
    df = rename_categories(df, category_mapping)

    # Merge categories
    df = merge_and_sum_categories(df, "Knallerter", "Cykler")
    df['KATEGORI'] = df['KATEGORI'].replace({"Cykler": "CYKLER + KNALLERTER"})
    df = merge_and_sum_categories(df, "CYKLER MOD ENSRETNINGEN", "CYKLER + KNALLERTER")
    df = merge_and_sum_categories(df, "EL-LØBEHJUL O.L.", "CYKLER + KNALLERTER")

    df = merge_and_sum_categories(df, "MOTORCYKLER", "PERSONBILER")  
    df = merge_and_sum_categories(df, "TAXA", "PERSONBILER")  
    df['KATEGORI'] = df['KATEGORI'].replace({"PERSONBILER": "PERSONBILER + MC"})

    df = merge_and_sum_categories(df, "LASTBILER, 2 AKSLER", "LASTBILER, 3 AKSLER")
    df = merge_and_sum_categories(df, "LASTBILER, 3 AKSLER", "LASTBILER, 4 - flere aksler")
    df = merge_and_sum_categories(df, "LASTBILER, 2-3 AKSLER", "LASTBILER, 4 - flere aksler")
    df['KATEGORI'] = df['KATEGORI'].replace({"LASTBILER, 4 - flere aksler": "LASTBILER"})

    df = merge_and_sum_categories(df, "ANDRE BUSSER", "BUSSER")
    df = merge_and_sum_categories(df, "BUSSER I FAST RUTE", "BUSSER")

    # Define the desired order
    category_order = [
        "PERSONBILER + MC",
        "VAREVOGNE (MAX. 3,5 t)",
        "A: LET TRAFIK I ALT",
        "LASTBILER",
        "BUSSER",
        "B: TUNG TRAFIK I ALT",
        "A+B: KØRETØJER I ALT",
        "LADCYKLER",
        "CYKLER + KNALLERTER",
        "CYKLER + KNALLERTER I ALT"
    ]

    # Total sum
    df = Total_sum(df)
    df = calculate_tung_traffic_percentage(df)
    df = calculate_Ladcykler_percentage(df)
    # Assign ranking index and sort
    df["sort_order"] = df["KATEGORI"].apply(lambda x: category_order.index(x) if x in category_order else len(category_order))
    df = df.sort_values(by=["TS", "ÅR", "sort_order"]).drop(columns=["sort_order"])
    
    df.to_csv(output_file_path, index=False)
    print(f"\nFile processed and saved successfully as {output_file_path}.")
    print(df['KATEGORI'].unique())

# Run the main function
input_file_path = 'Original udtræk.csv'
output_file_path = 'Cleaned_data.csv'
main(input_file_path, output_file_path)



File processed and saved successfully as Cleaned_data.csv.
['PERSONBILER + MC' 'VAREVOGNE (MAX. 3,5 t)' 'A: LET TRAFIK I ALT'
 'LASTBILER' 'B: TUNG TRAFIK I ALT' 'A+B: KØRETØJER I ALT' 'LADCYKLER'
 'CYKLER + KNALLERTER' 'CYKLER + KNALLERTER I ALT' 'BUSSER']
